In [5]:
# Some imports for handling images
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image

# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."


def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")    


def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image


price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)    

OpenAI API Key exists and begins sk-proj-


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [1]:
# Some imports for handling images
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
import datetime

# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Set this to False to disable image generation and avoid DALL-E charges
ENABLE_IMAGES = False

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."


def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))


import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))
  

# Define the get_ticket_price function (adding this as it's referenced but not defined in original)
def get_ticket_price(city):
    # Simple price calculation for demo purposes
    # In a real application, this would query a database or API
    prices = {
        "new york": 450,
        "london": 850,
        "paris": 900,
        "tokyo": 1200,
        "sydney": 1500,
        "dubai": 950,
        "rome": 880,
        "singapore": 1100,
    }
    return prices.get(city.lower(), 999)  # Default price if city not found

# Function to save booking details to a text file
def book_ticket(name, destination, departure_date, return_date, passengers):
    booking_id = f"FLT{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    booking_info = {
        "booking_id": booking_id,
        "name": name,
        "destination": destination,
        "departure_date": departure_date,
        "return_date": return_date,
        "passengers": passengers,
        "price_per_ticket": get_ticket_price(destination),
        "total_price": get_ticket_price(destination) * int(passengers),
        "booking_date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Create a bookings directory if it doesn't exist
    os.makedirs("bookings", exist_ok=True)
    
    # Write booking details to a text file
    filename = f"bookings/{booking_id}.txt"
    with open(filename, "w") as f:
        f.write(f"Booking ID: {booking_info['booking_id']}\n")
        f.write(f"Passenger Name: {booking_info['name']}\n")
        f.write(f"Destination: {booking_info['destination']}\n")
        f.write(f"Departure Date: {booking_info['departure_date']}\n")
        f.write(f"Return Date: {booking_info['return_date']}\n")
        f.write(f"Number of Passengers: {booking_info['passengers']}\n")
        f.write(f"Price per Ticket: ${booking_info['price_per_ticket']}\n")
        f.write(f"Total Price: ${booking_info['total_price']}\n")
        f.write(f"Booking Date: {booking_info['booking_date']}\n")
    
    return booking_info


def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        for tool_call in message.tool_calls:
            tool_response, city = handle_tool_call(tool_call)
            messages.append(message)
            messages.append(tool_response)
            if ENABLE_IMAGES and city:  # Only generate image if ENABLE_IMAGES is True and a city was provided
                image = artist(city)
                
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image


price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "book_ticket",
    "description": "Book a ticket for the customer. Call this when a customer wants to book a flight.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Name of the passenger",
            },
            "destination": {
                "type": "string",
                "description": "The destination city",
            },
            "departure_date": {
                "type": "string",
                "description": "Date of departure in YYYY-MM-DD format",
            },
            "return_date": {
                "type": "string",
                "description": "Date of return in YYYY-MM-DD format",
            },
            "passengers": {
                "type": "string",
                "description": "Number of passengers",
            }
        },
        "required": ["name", "destination", "departure_date", "return_date", "passengers"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

# Updated handle_tool_call function to work with both tools
def handle_tool_call(tool_call):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    city = None
    
    if function_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response_content = json.dumps({"destination_city": city, "price": price})
    
    elif function_name == "book_ticket":
        name = arguments.get('name')
        destination = arguments.get('destination')
        departure_date = arguments.get('departure_date')
        return_date = arguments.get('return_date')
        passengers = arguments.get('passengers')
        
        booking_info = book_ticket(name, destination, departure_date, return_date, passengers)
        response_content = json.dumps(booking_info)
        city = destination  # Set city for image generation
    
    response = {
        "role": "tool",
        "content": response_content,
        "tool_call_id": tool_call.id
    }
    
    return response, city

# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

OpenAI API Key exists and begins sk-proj-
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [3]:
# Imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
import datetime
from IPython.display import Audio, display

# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

ENABLE_IMAGES = True

system_message = "You are a helpful assistant for an Airline called FlightAI. " \
                 "Give short, courteous answers, no more than 1 sentence. " \
                 "Always be accurate. If you don't know the answer, say so."

# Image generation
def artist(city):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

# Text-to-speech
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    display(Audio(output_filename, autoplay=True))

# Ticket price lookup
def get_ticket_price(city):
    prices = {
        "new york": 450, "london": 850, "paris": 900,
        "tokyo": 1200, "sydney": 1500, "dubai": 950,
        "rome": 880, "singapore": 1100
    }
    return prices.get(city.lower(), 999)

# Booking function
def book_ticket(name, destination, departure_date, return_date, passengers):
    booking_id = f"FLT{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
    booking_info = {
        "booking_id": booking_id,
        "name": name,
        "destination": destination,
        "departure_date": departure_date,
        "return_date": return_date,
        "passengers": passengers,
        "price_per_ticket": get_ticket_price(destination),
        "total_price": get_ticket_price(destination) * int(passengers),
        "booking_date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    os.makedirs("bookings", exist_ok=True)
    filename = f"bookings/{booking_id}.txt"
    with open(filename, "w") as f:
        for key, value in booking_info.items():
            f.write(f"{key.replace('_', ' ').title()}: {value}\n")

    return booking_info

# Chat + Tool use
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        for tool_call in message.tool_calls:
            tool_response, city = handle_tool_call(tool_call)
            messages.append(message)
            messages.append(tool_response)
            if ENABLE_IMAGES and city:
                image = artist(city)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]
    talker(reply)
    return history, image

# Function calling definitions
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "Destination city name"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "book_ticket",
    "description": "Book a flight ticket",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "destination": {"type": "string"},
            "departure_date": {"type": "string"},
            "return_date": {"type": "string"},
            "passengers": {"type": "string"}
        },
        "required": ["name", "destination", "departure_date", "return_date", "passengers"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

# Tool call handler
def handle_tool_call(tool_call):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    city = None

    if function_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response_content = json.dumps({"destination_city": city, "price": price})

    elif function_name == "book_ticket":
        booking_info = book_ticket(
            arguments["name"],
            arguments["destination"],
            arguments["departure_date"],
            arguments["return_date"],
            arguments["passengers"]
        )
        response_content = json.dumps(booking_info)
        city = arguments["destination"]

    response = {
        "role": "tool",
        "content": response_content,
        "tool_call_id": tool_call.id
    }

    return response, city

# Translation function
def translate_prompt(prompt, lang_code):
    if lang_code == "en":
        return prompt
    translation_prompt = f"Translate the following to {lang_code}:\n{prompt}"
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a translator."},
            {"role": "user", "content": translation_prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Gradio UI
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Type your message:")
            language_dropdown = gr.Dropdown(
                choices=["en", "fr", "es", "de", "ar", "zh", "hi"],
                value="en",
                label="Translate To Language (use 'en' for English)"
            )
            translated_box = gr.Textbox(label="Translated Prompt (auto)", interactive=False)

        clear_button = gr.Button("Clear")

    def process_user_input(message, lang, history):
        translated = translate_prompt(message, lang)
        history += [{"role": "user", "content": translated}]
        return "", history, translated

    user_input.submit(process_user_input, inputs=[user_input, language_dropdown, chatbot],
                      outputs=[user_input, chatbot, translated_box]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

    clear_button.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)


OpenAI API Key exists and begins sk-proj-
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
